# Predicting using BigQuery ML & XGBoost

This notebook illustrates:
<ol>
<li> Machine Learning using BigQuery
<li> Jupyter Magic for BigQuery in Cloud Datalab
</ol>

Please see [this notebook](1_explore.ipynb) for more context on this problem and how the features were chosen.

In [ ]:
# change these to try this notebook out
PROJECT = 'data-science-showroom'
REGION = 'europe-west1'

In [ ]:
import os
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## Exploring the Data

Here, we will be taking natality data and training on features to predict the birth weight.

The CDC's Natality data has details on US births from 1969 to 2008 and is available in BigQuery as a public data set. More details: https://bigquery.cloud.google.com/table/publicdata:samples.natality?tab=details

Lets start by looking at the data since 2000 with useful values > 0!

In [ ]:
%%bigquery df
WITH DATA AS
(SELECT 
       MOD(JOBID,4) AS HASH_JOBID, VERSION, PERFORMANCE, PRECISION, MPLINK, NTNU, MPLINK_NTNU,
       MBS, RUNEND, TIMESTEP, NBNODES, NBELEM1D, NBELEM2D,
       NBELEM3D, CLUSTER, NBSERVERS, NBCORE, DATACHECK_TIME,
       ELAPSEDTIME
FROM
  `challenge.training_data`)

SELECT * FROM DATA WHERE HASH_JOBID >0

In [ ]:
df.head()

## Train Model

With the relevant columns chosen to accomplish predictions, it is then possible to create (train) the model in BigQuery. First, a dataset will be needed store the model. (if this throws an error in Datalab, simply create the dataset from the BigQuery console).

In [ ]:
%%bash
bq --location=EU mk -d demo_youripn

With the demo dataset ready, it is possible to create a linear regression model to train the model.

This will take approximately **4 minutes** to run and will show **Done** when complete.

In [ ]:
%%bigquery model
CREATE or REPLACE MODEL demo_youripn.xgboost_model
TRANSFORM( ML.STANDARD_SCALER(PRECISION) OVER() AS PRECISION,
           ML.STANDARD_SCALER(RUNEND) OVER() AS RUNEND,
           ML.STANDARD_SCALER(TIMESTEP) OVER() AS TIMESTEP,
           ML.STANDARD_SCALER(NBNODES) OVER() AS NBNODES,
           ML.STANDARD_SCALER(NBELEM1D) OVER() AS NBELEM1D,
           ML.STANDARD_SCALER(NBELEM2D) OVER() AS NBELEM2D,
           ML.STANDARD_SCALER(NBELEM3D) OVER() AS NBELEM3D,
           ML.STANDARD_SCALER(NBSERVERS) OVER() AS NBSERVERS,
           ML.STANDARD_SCALER(NBCORE) OVER() AS NBCORE,
           ML.STANDARD_SCALER(DATACHECK_TIME) OVER() AS DATACHECK_TIME,
           VERSION,
           PERFORMANCE,
           MPLINK,
           NTNU,
           MPLINK_NTNU,
           MBS,
           CLUSTER,
           ML.FEATURE_CROSS(STRUCT(PERFORMANCE, MPLINK, NTNU, CLUSTER)) AS CROSS_F,
           ELAPSEDTIME)
OPTIONS(MODEL_TYPE='BOOSTED_TREE_REGRESSOR',
        BOOSTER_TYPE = 'GBTREE',
        L1_REG = 0.1,
        L2_REG = 0.01,
        NUM_PARALLEL_TREE = 1,
        MAX_ITERATIONS = 30,
        TREE_METHOD = 'HIST',
        EARLY_STOP = FALSE,
        SUBSAMPLE = 0.85,
        INPUT_LABEL_COLS = ['ELAPSEDTIME'])
AS 
SELECT 
       *
FROM
  `challenge.training_data`
WHERE MOD(JOBID,4)<3 

## Training Statistics

During the model training (and after the training), it is possible to see the model's training evaluation statistics.

For each training run, a table named `<model_name>_eval` is created. This table has basic performance statistics for each iteration.

While the new model is training, review the training statistics in the BigQuery UI to see the below model training: https://bigquery.cloud.google.com/

Since these statistics are updated after each iteration of model training, you will see different values for each refresh while the model is training.

The training details may also be viewed after the training completes from this notebook.

In [ ]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL demo_youripn.xgboost_model);

Some of these columns are obvious although what do the non-specific ML columns mean (specific to BQML)?

**training_run** - Will be zero for a newly created model. If the model is re-trained using warm_start, this will increment for each re-training.

**iteration** - Number of the associated `training_run`, starting with zero for the first iteration.

**duration_ms** - Indicates how long the iteration took (in ms).

Note: You can also see these stats by refreshing the BigQuery UI window, finding the `<model_name>` table, selecting on it, and then the Training Stats sub-header.

Let's plot the training and evaluation loss to see if the model has an overfit.

As you can see, the training loss and evaluation loss are essentially identical. We do not seem to be overfitting.

## Model Evaluation with BQML using the Model

With a trained model, it is now possible to make a prediction on the values. The only difference from the second query above is the reference to the model. The data has been limited (`LIMIT 100`) to reduce amount of data returned.

When the `ml.predict` function is leveraged, output prediction column name for the model is `predicted_<label_column_name>`.

In [ ]:
%%bigquery
SELECT
  *
FROM
  ml.EVALUATE(MODEL demo_youripn.xgboost_model,
(SELECT 
    *
FROM
  `challenge.training_data`
WHERE MOD(JOBID,4) =3))

In [ ]:
%%bigquery data
SELECT
  *
FROM
  ml.PREDICT(MODEL demo_youripn.xgboost_model,
(SELECT 
    *
FROM
  `challenge.competition`
WHERE MOD(JOBID,4) =0))
    LIMIT 1000